# 06_데이터프레임 병합과 연결
### 1. merge()를 이용한 데이터프레임 병합
- **merge()** : 두 개 데이터프레임의 공통 열 또는 공통 인덱스를 기준으로 두 개의
데이터프레임을 하나로 합침
        DataFrame.merge(right, how='inner', on=None, left_on=None,
        right_on=None, left_index=False, right_index=False, sort=False,
        suffixes=('_x', '_y'), copy=None, indicator=False, validate=None)
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html

##### 1) how 속성
- how 매개변수의 기본값은‘inner’이므로 내부 조인을 함

In [1]:
import pandas as pd

In [2]:
df1 = pd.DataFrame({'key': ['a','b','c','f'],
                    'c1': [1,2,3,5]})

In [3]:
df2 = pd.DataFrame({'key': ['a','b','d','f'],
                    'c2': [5,6,7,8]})

In [4]:
df1.merge(df2)  # how 속성의 기본값은 inner -> 내부 조인

,key,c1,c2
0,a,1,5
1,b,2,6
2,f,5,8


In [5]:
df1.merge(df2, how='left')  # left 조인

,key,c1,c2
0,a,1,5.0
1,b,2,6.0
2,c,3,NaN
3,f,5,8.0


In [6]:
df1.merge(df2, how='right')  # right 조인

,key,c1,c2
0,a,1.0,5
1,b,2.0,6
2,d,NaN,7
3,f,5.0,8


In [7]:
df1.merge(df2, how='outer')  # outer 조인

,key,c1,c2
0,a,1.0,5.0
1,b,2.0,6.0
2,c,3.0,NaN
3,d,NaN,7.0
4,f,5.0,8.0


##### 2) left_on, right_on 속성
- 키가 되는 기준 열의 이름이 두 데이터프레임에서 다르다면 left_on과 right_on인수를 사용하여 기준이 되는 열을 명시해야 함

In [8]:
df3 = pd.DataFrame({'key3': ['a','b','c','f'],
                    'c1': [1,2,3,5]})

In [9]:
df4 = pd.DataFrame({'key4': ['a','b','d','f'],
                    'c2': [5,6,7,8]})

In [10]:
df3.merge(df4)  # 공통열이 없어서 오류 발생

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False

In [11]:
df3.merge(df4, left_on='key3', right_on='key4')

,key3,c1,key4,c2
0,a,1,a,5
1,b,2,b,6
2,f,5,f,8


In [12]:
df3.merge(df4, left_index=True, right_index=True)  # 인덱스(행이름)를 기준으로 정렬

,key3,c1,key4,c2
0,a,1,a,5
1,b,2,b,6
2,c,3,d,7
3,f,5,f,8


### 2. concat()을 이용한 데이터프레임 연결
- **concat()** : 두 데이터프레임을 연결함
        pandas.concat(objs, *, axis=0, join='outer', ignore_index=False,
        keys=None, levels=None, names=None, verify_integrity=False, sort=False, copy=None)
https://pandas.pydata.org/docs/reference/api/pandas.concat.html#pandas.concat

##### 1) axis 속성
- concat()은 axis 속성에 따라  
  axis=0 이면 데이터프레임을 위에서 아래로 연결하며,  
  axis=1이면 왼쪽에서 오른쪽으로 연결

In [13]:
df1 = pd.DataFrame({'c1': [1,2,3,4],
                    'c2': [5,6,7,8]})

In [14]:
df2 = pd.DataFrame({'c3': ['a','b','c','d'],
                    'c4': [1.2, 3.4, 5.5, 7.6]})

In [15]:
pd.concat([df1, df2], axis=1)

,c1,c2,c3,c4
0,1,5,a,1.2
1,2,6,b,3.4
2,3,7,c,5.5
3,4,8,d,7.6


##### 2) axis=0 일 경우 주의사항
- axis=0일 경우 같은 이름의 열 이름이 있을 경우 위에서 아래로 데이터를 쌓아 연결해 주지만  
 열 이름이 다를 경우 없는 곳의 데이터가 NaN 값이 됨

In [16]:
pd.concat([df1, df2], axis=0)

,c1,c2,c3,c4
0,1.0,5.0,NaN,NaN
1,2.0,6.0,NaN,NaN
2,3.0,7.0,NaN,NaN
3,4.0,8.0,NaN,NaN
0,NaN,NaN,a,1.2
1,NaN,NaN,b,3.4
2,NaN,NaN,c,5.5
3,NaN,NaN,d,7.6


### 3. concat()과 reset_index()
- 연결할 때 내부적으로 인덱스를 사용하므로 **두 데이터프레임의 인덱스가 다를 경우**에는 **reset_index()**를 이용해서 인덱스를 초기화 해줘야함
        DataFrame.reset_index(level=None, *, drop=False, inplace=False, col_level=0, col_fill='')
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.reset_index.html#pandas.DataFrame.reset_index

In [17]:
df1 = pd.DataFrame({'c1': [1,2,3,4],
                    'c2': [5,6,7,8]}, index=[0,2,4,6])

In [18]:
df2 = pd.DataFrame({'c3': ['a','b','c','d'],
                    'c4': [1.2, 3.4, 5.5, 7.6]}, index=[0,1,2,3])

In [19]:
df1

,c1,c2
0,1,5
2,2,6
4,3,7
6,4,8


In [20]:
df2

,c3,c4
0,a,1.2
1,b,3.4
2,c,5.5
3,d,7.6


In [21]:
pd.concat([df1, df2], axis=1, sort=True)

,c1,c2,c3,c4
0,1.0,5.0,a,1.2
1,NaN,NaN,b,3.4
2,2.0,6.0,c,5.5
3,NaN,NaN,d,7.6
4,3.0,7.0,NaN,NaN
6,4.0,8.0,NaN,NaN


In [22]:
df1.reset_index()

,index,c1,c2
0,0,1,5
1,2,2,6
2,4,3,7
3,6,4,8


In [23]:
df1.reset_index(drop=True)  # 기존 인덱스를 제거

,c1,c2
0,1,5
1,2,6
2,3,7
3,4,8


In [24]:
# 출력이 됐다는 것은 현재 객체가 바뀌지 않았음을 의미
df1.reset_index(drop=True, inplace=True)  # inplace=True : 현재 객체를 변경

In [25]:
df1

,c1,c2
0,1,5
1,2,6
2,3,7
3,4,8


In [26]:
df2

,c3,c4
0,a,1.2
1,b,3.4
2,c,5.5
3,d,7.6


In [27]:
pd.concat([df1, df2], axis=1)

,c1,c2,c3,c4
0,1,5,a,1.2
1,2,6,b,3.4
2,3,7,c,5.5
3,4,8,d,7.6


[정리]

    - merge() : 병합, 내부조인, 외부조인 -> how 속성을 이용해서 설정
    - concat() : 연결, 인덱스(행이름)를 기준으로 연결